In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [3]:
def load_data(file_path):
    """
    Load data from a CSV file.
    """
    return pd.read_csv(file_path)

In [ ]:
def split_inputs_output(data, inputs, output):
    """
    Split features and target variables.
    """
    X = data[inputs]
    y = data[output]
    return X, y

In [2]:
pip install prefect

  Using cached prefect-2.16.6-py3-none-any.whl.metadata (10 kB)
  Using cached aiosqlite-0.20.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached apprise-1.7.4-py3-none-any.whl.metadata (44 kB)
  Using cached dateparser-1.2.0-py2.py3-none-any.whl.metadata (28 kB)
  Using cached docker-6.1.3-py3-none-any.whl.metadata (3.5 kB)
  Using cached graphviz-0.20.3-py3-none-any.whl.metadata (12 kB)
  Using cached griffe-0.42.1-py3-none-any.whl.metadata (6.2 kB)
  Using cached kubernetes-29.0.0-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached readchar-4.0.6-py3-none-any.whl.metadata (6.2 kB)
  Using cached typer-0.11.0-py3-none-any.whl.metadata (13 kB)
  Using cached asgi_lifespan-2.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached cachetools-5.3.3-py3-none-any.whl.metadata (5.3 kB)
  Using cached coolname-2.2.0-py2.py3-none-any.whl.metadata (6.2 kB)
  Using cached croniter-2.0.3-py2.py3-none-any.whl.metadata (25 kB)
  Using cached importlib_resources-6.1.3-py3-none-any.whl.metadata (3.9 

In [29]:
df=pd.read_csv('data.csv')
df.dropna(inplace=True)

In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from nltk.corpus import stopwords
from prefect import flow, task
from sklearn.naive_bayes import MultinomialNB
import re
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
la=LabelEncoder()

@task
def load_dat(file_path):
    """
    Load data from a CSV file.
    """
    return pd.read_csv(file_path)


@task
def split_inputs_outpu(data, inputs, output):
    """
    Split features and target variables.
    """
    X = data[inputs]
    y = data[output]
    return X, y
	

@task
def split_train_te(X, y, test_size=0.25, random_state=0):
    """
    Split data into train and test sets.
    """
    return train_test_split(X, y, test_size=test_size, random_state=random_state)
	
	
@task
def preprocess_da(X_train, X_test, y_train, y_test):
    """
    Rescale the data.
    """
    tfid=TfidfVectorizer()
    X_train_id = tfid.fit_transform(X_train)
    X_test_id = tfid.transform(X_test)
    return X_train_id, X_test_id, y_train, y_test
	

@task
def train_mod(X_train_scaled, y_train):
    """
    Training the machine learning model.
    """
    clf = MultinomialNB()
    clf.fit(X_train_scaled, y_train)
    return clf
	

@task
def evaluate_mod(model, X_train_scaled, y_train, X_test_scaled, y_test):
    """
    Evaluating the model.
    """
    y_train_pred = model.predict(X_train_scaled)
    y_test_pred = model.predict(X_test_scaled)

    train_score = metrics.accuracy_score(y_train, y_train_pred)
    test_score = metrics.accuracy_score(y_test, y_test_pred)
    
    return train_score, test_score

@task
def calculate(ratings):
    l=[]
    for i in range(len(df['Ratings'])):
        if int(df['Ratings'].iloc[i]) >= 4:
            l.append('positive')
        else:
            l.append('negative')
    return l

@task
def preprocess_(text):
    text = str(text)
    sentence = re.sub("[^a-zA-Z]", " ", text)
    sentence = sentence.lower()
    tokens = sentence.split()
    tokens = [t for t in tokens if not t in stopwords.words("english")]
    tokens = [lemmatizer.lemmatize(word,pos='v') for word in tokens]
    demo=pd.Series([" ".join(tokens), len(tokens)])
    return demo[0]


# Workflow
@flow(name="random nb")
def workflow():
    DATA_PATH = "data.csv"
    INPUTS = 'Review text'
    OUTPUT = 'Ratings'
    parameters = {
    'tfidf__max_df': (0.25, 0.5, 0.75),
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'nb__alpha': (0.1, 0.5, 1)
}
    # Load data
    sentiment = load_data1(DATA_PATH)

    # Identify Inputs and Output
    X=df['Review text']
    val=calculate(df['Ratings'])
    y=la.fit_transform(val)
    # Split data into train and test sets
    X_train, X_test, y_train, y_test = split_train_te(X, y)

    # Preprocess the data
    X_train_scaled, X_test_scaled, y_train, y_test = preprocess_da(X_train, X_test, y_train, y_test)

    # Build a model
    model = train_mod(X_train_scaled, y_train)
    
    # Evaluation
    train_score, test_score = evaluate_mod(model, X_train_scaled, y_train, X_test_scaled, y_test)
    
    print("Train Score:", train_score)
    print("Test Score:", test_score)



if __name__ == "__main__":
    workflow()

c:\Users\sspon\anaconda3\New folder\Lib\site-packages\prefect\tasks.py:348: UserWarning: A task named 'load_dat' and defined at 'C:\Users\sspon\AppData\Local\Temp\ipykernel_6720\4101147067.py:14' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
c:\Users\sspon\anaconda3\New folder\Lib\site-packages\prefect\tasks.py:348: UserWarning: A task named 'split_inputs_outpu' and defined at 'C:\Users\sspon\AppData\Local\Temp\ipykernel_6720\4101147067.py:22' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
c:\Users\sspon\anaconda3\New folder\Lib\site-packages\prefect\tasks.py:348: UserWarning: A task named 'split_train_te' and defined at 'C:\Users\sspon\AppData\Local\Temp\ipykernel_6720\4101147067.py:32' conflicts with another task. Consider specifying a unique `name` parameter in the task d

11:57:30.664 | INFO    | prefect.engine - Created flow run 'primitive-jaguarundi' for flow 'random nb'

11:57:30.914 | INFO    | Flow run 'primitive-jaguarundi' - Created task run 'load_data1-0' for task 'load_data1'

11:57:30.914 | INFO    | Flow run 'primitive-jaguarundi' - Executing 'load_data1-0' immediately...

11:57:31.231 | INFO    | Task run 'load_data1-0' - Finished in state Completed()

...

11:57:34.135 | INFO    | Flow run 'primitive-jaguarundi' - Created task run 'evaluate_mod-0' for task 'evaluate_mod'

11:57:34.138 | INFO    | Flow run 'primitive-jaguarundi' - Executing 'evaluate_mod-0' immediately...

11:57:34.356 | INFO    | Task run 'evaluate_mod-0' - Finished in state Completed()

Train Score: 0.8894990847062739
Test Score: 0.8622754491017964


11:57:34.493 | INFO    | Flow run 'primitive-jaguarundi' - Finished in state Completed('All states completed.')

In [7]:
from prefect import flow, task

@task
def print_hello(name):
    print(f"Hello {name}!")

@flow(name="Hello Flow")
def hello_world(name="world"):
    print_hello(name)

In [15]:
import httpx
from prefect import flow


@flow
def get_repo_info():
    url = "https://api.github.com/repos/PrefectHQ/prefect"
    response = httpx.get(url)
    response.raise_for_status()
    repo = response.json()
    print("PrefectHQ/prefect repository statistics 🤓:")
    print(f"Stars 🌠 : {repo['stargazers_count']}")
    print(f"Forks 🍴 : {repo['forks_count']}")

if __name__ == "__main__":
    get_repo_info()

c:\Users\sspon\anaconda3\New folder\Lib\site-packages\prefect\flows.py:357: UserWarning: A flow named 'get-repo-info' and defined at 'C:\Users\sspon\AppData\Local\Temp\ipykernel_6720\961705441.py:5' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


11:11:04.286 | INFO    | prefect.engine - Created flow run 'natural-tanuki' for flow 'get-repo-info'

PrefectHQ/prefect repository statistics 🤓:
Stars 🌠 : 14359
Forks 🍴 : 1431


11:11:05.734 | INFO    | Flow run 'natural-tanuki' - Finished in state Completed()

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier

from sklearn import metrics

from prefect import task, flow

@task
def load_data3(file_path):
    """
    Load data from a CSV file.
    """
    return pd.read_csv(file_path)


@task
def split_inputs_output3(data, inputs, output):
    """
    Split features and target variables.
    """
    X = data[inputs]
    y = data[output]
    return X, y
	

@task
def split_train_test3(X, y, test_size=0.25, random_state=0):
    """
    Split data into train and test sets.
    """
    return train_test_split(X, y, test_size=test_size, random_state=random_state)
	
	
@task
def preprocess_data3(X_train, X_test, y_train, y_test):
    """
    Rescale the data.
    """
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled, y_train, y_test
	

@task
def train_model3(X_train_scaled, y_train, hyperparameters):
    """
    Training the machine learning model.
    """
    clf = KNeighborsClassifier(**hyperparameters)
    clf.fit(X_train_scaled, y_train)
    return clf
	

@task
def evaluate_model3(model, X_train_scaled, y_train, X_test_scaled, y_test):
    """
    Evaluating the model.
    """
    y_train_pred = model.predict(X_train_scaled)
    y_test_pred = model.predict(X_test_scaled)

    train_score = metrics.accuracy_score(y_train, y_train_pred)
    test_score = metrics.accuracy_score(y_test, y_test_pred)
    
    return train_score, test_score


# Workflow
@flow(name="variable1")
def workflow():
    DATA_PATH = "iris.csv"
    INPUTS = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']
    OUTPUT = 'Species'
    HYPERPARAMETERS = {'n_neighbors': 3, 'p': 2}
    
    # Load data
    iris = load_data3(DATA_PATH)

    # Identify Inputs and Output
    X, y = split_inputs_output3(iris, INPUTS, OUTPUT)

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = split_train_test3(X, y)

    # Preprocess the data
    X_train_scaled, X_test_scaled, y_train, y_test = preprocess_data3(X_train, X_test, y_train, y_test)

    # Build a model
    model = train_model3(X_train_scaled, y_train, HYPERPARAMETERS)
    
    # Evaluation
    train_score, test_score = evaluate_model3(model, X_train_scaled, y_train, X_test_scaled, y_test)
    
    print("Train Score:", train_score)
    print("Test Score:", test_score)



if __name__ == "__main__":
    workflow()

11:13:06.546 | INFO    | prefect.engine - Created flow run 'paper-warthog' for flow 'variable1'

11:13:06.789 | INFO    | Flow run 'paper-warthog' - Created task run 'load_data3-0' for task 'load_data3'

11:13:06.789 | INFO    | Flow run 'paper-warthog' - Executing 'load_data3-0' immediately...

11:13:07.186 | INFO    | Task run 'load_data3-0' - Finished in state Completed()

11:13:07.314 | INFO    | Flow run 'paper-warthog' - Created task run 'split_inputs_output3-0' for task 'split_inputs_output3'

...

11:13:08.789 | INFO    | Flow run 'paper-warthog' - Created task run 'evaluate_model3-0' for task 'evaluate_model3'

11:13:08.797 | INFO    | Flow run 'paper-warthog' - Executing 'evaluate_model3-0' immediately...

11:13:09.230 | INFO    | Task run 'evaluate_model3-0' - Finished in state Completed()

Train Score: 0.9732142857142857
Test Score: 0.9736842105263158


11:13:09.337 | INFO    | Flow run 'paper-warthog' - Finished in state Completed('All states completed.')